# Description

TODO

# Modules

In [1]:
import re

import pandas as pd
# from scipy import stats
# import seaborn as sns

# from clustermatch.plots import plot_histogram, plot_cumulative_histogram, jointplot
from clustermatch import conf

# Settings

In [2]:
# DATASET_CONFIG = conf.GTEX
# GTEX_TISSUE = "whole_blood"
# GENE_SEL_STRATEGY = "var_pc_log2"

In [3]:
# # this is used for the cumulative histogram
# GENE_PAIRS_PERCENT = 0.70

In [4]:
# CLUSTERMATCH_LABEL = "Clustermatch"
# PEARSON_LABEL = "Pearson"
# SPEARMAN_LABEL = "Spearman"

In [5]:
GENE_FILE_MARK_TEMPLATE = "| *{gene}* |"

In [6]:
GENE0_STATS_TEMPLATE = '| *{gene}* | ADD DIRECT<!-- $rowspan="2" --> | {blood_min} | {blood_avg} | {blood_max} | ADD DIRECT<!-- $rowspan="2" --> | {pred_min} | {pred_avg} | {pred_max} |'
GENE1_STATS_TEMPLATE = '| *{gene}* | {blood_min} | {blood_avg} | {blood_max} | {pred_min} | {pred_avg} | {pred_max}<!-- $removenext="3" --> |'

# Paths

In [7]:
assert (
    conf.MANUSCRIPT["BASE_DIR"] is not None
), "The manuscript directory was not configured"

In [8]:
OUTPUT_FILE_PATH = conf.MANUSCRIPT["CONTENT_DIR"] / "20.00.supplementary_material.md"
display(OUTPUT_FILE_PATH)
assert OUTPUT_FILE_PATH.exists()

PosixPath('/opt/manuscript/content/20.00.supplementary_material.md')

In [9]:
# assert (
#     conf.MANUSCRIPT["BASE_DIR"] is not None and conf.MANUSCRIPT["BASE_DIR"].exists()
# ), "Manuscript dir not set"

In [10]:
# OUTPUT_FIGURE_DIR = (
#     conf.MANUSCRIPT["FIGURES_DIR"] / "coefs_comp" / f"gtex_{GTEX_TISSUE}"
# )
# OUTPUT_FIGURE_DIR.mkdir(parents=True, exist_ok=True)
# display(OUTPUT_FIGURE_DIR)

In [11]:
# INPUT_CORR_FILE_TEMPLATE = (
#     DATASET_CONFIG["SIMILARITY_MATRICES_DIR"]
#     / DATASET_CONFIG["SIMILARITY_MATRIX_FILENAME_TEMPLATE"]
# )
# display(INPUT_CORR_FILE_TEMPLATE)

In [12]:
# INPUT_FILE = DATASET_CONFIG["SIMILARITY_MATRICES_DIR"] / str(
#     INPUT_CORR_FILE_TEMPLATE
# ).format(
#     tissue=GTEX_TISSUE,
#     gene_sel_strategy=GENE_SEL_STRATEGY,
#     corr_method="all",
# )
# display(INPUT_FILE)

# assert INPUT_FILE.exists()

In [13]:
INPUT_DIR = conf.GIANT["RESULTS_DIR"]
display(INPUT_DIR)

assert INPUT_DIR.exists()

PosixPath('/opt/data/results/giant')

# Functions

In [14]:
def read_data(gene0, gene1, tissue_name, mode="visible"):
    input_filename = f"{gene0.lower()}_{gene1.lower()}-network-{mode}.csv"

    input_filepath = INPUT_DIR / tissue_name / input_filename
    assert input_filepath.exists()

    data = pd.read_csv(input_filepath)
    
    assert (
        ((gene0 in data["GENE1"].unique()) or (gene0 in data["GENE2"].unique())) and
        ((gene1 in data["GENE1"].unique()) or (gene1 in data["GENE2"].unique()))
    )
    
    return data

In [15]:
# testing
_tmp0 = read_data("IFNG", "SDS", "blood")
display(_tmp0.shape)

_tmp1 = read_data("IFNG", "SDS", "pred")
display(_tmp1.shape)

assert _tmp0.shape[0] != _tmp1.shape[0]

(127, 3)

(124, 3)

In [16]:
def format_number(number):
    return f"{number:.2f}"

In [17]:
# testing
format_number(0.222222) == "0.22"
format_number(0.225222) == "0.23"

True

In [18]:
def get_gene_stats(df, gene_name):
    gene_data = df[(df["GENE1"] == gene_name) | (df["GENE2"] == gene_name)]
    return gene_data.describe().squeeze()

In [19]:
# testing
_tmp0_stats = get_gene_stats(_tmp0, "IFNG")
assert _tmp0_stats["min"].round(2) == 0.19
assert _tmp0_stats["mean"].round(2) == 0.42
assert _tmp0_stats["max"].round(2) == 0.54

In [20]:
def get_gene_content(blood_stats, pred_stats, gene_name, gene_template):
    return gene_template.format(
        gene=gene_name,
        blood_min=format_number(blood_stats["min"]),
        blood_avg=format_number(blood_stats["mean"]),
        blood_max=format_number(blood_stats["max"]),
        pred_min=format_number(pred_stats["min"]),
        pred_avg=format_number(pred_stats["mean"]),
        pred_max=format_number(pred_stats["max"]),
    )

In [21]:
# testing
_tmp_gene_cont = get_gene_content(_tmp0_stats, _tmp0_stats, "IFNG", GENE0_STATS_TEMPLATE)
assert "IFNG" in _tmp_gene_cont
assert "0.19" in _tmp_gene_cont
assert "0.42" in _tmp_gene_cont
assert "0.54" in _tmp_gene_cont

In [22]:
def write_content(text, text_replacement):
    with open(OUTPUT_FILE_PATH, "r", encoding="utf8") as f:
        file_content = f.read()

    new_file_content = re.sub(
        re.escape(text) + ".+\n",
        text_replacement,
        file_content,
        # flags=re.DOTALL,
    )

    with open(OUTPUT_FILE_PATH, "w", encoding="utf8") as f:
        f.write(new_file_content)

In [23]:
def process_genes(gene0, gene1):
    data_blood = read_data(gene0, gene1, "blood")
    data_pred = read_data(gene0, gene1, "pred")

    for gene_name, gene_template in ((gene0, GENE0_STATS_TEMPLATE), (gene1, GENE1_STATS_TEMPLATE)):
        blood_stats = get_gene_stats(data_blood, gene_name).rename(f"{gene_name} - blood")
        display(blood_stats)

        pred_stats = get_gene_stats(data_pred, gene_name).rename(f"{gene_name} - pred")
        display(pred_stats)

        new_content = get_gene_content(blood_stats, pred_stats, gene_name, gene_template) + "\n"

        gene_file_mark = GENE_FILE_MARK_TEMPLATE.format(gene=gene_name)

        write_content(gene_file_mark, new_content)

# IFNG - SDS

In [24]:
process_genes("IFNG", "SDS")

count    15.000000
mean      0.416329
std       0.084663
min       0.186702
25%       0.384790
50%       0.416825
75%       0.466691
max       0.539424
Name: IFNG - blood, dtype: float64

count    15.000000
mean      0.895806
std       0.087539
min       0.742973
25%       0.831290
50%       0.920374
75%       0.972659
max       0.990921
Name: IFNG - pred, dtype: float64

count    7.000000
mean     0.287983
std      0.073519
min      0.180309
25%      0.253745
50%      0.274899
75%      0.319880
max      0.413422
Name: SDS - blood, dtype: float64

count    13.000000
mean      0.805204
std       0.099726
min       0.645825
25%       0.762556
50%       0.812201
75%       0.867130
max       0.944408
Name: SDS - pred, dtype: float64

# JUN - APOC1

In [25]:
process_genes("JUN", "APOC1")

count    15.000000
mean      0.679274
std       0.226874
min       0.262018
25%       0.516277
50%       0.728343
75%       0.857155
max       0.973947
Name: JUN - blood, dtype: float64

count    15.000000
mean      0.727964
std       0.183223
min       0.360925
25%       0.599624
50%       0.735264
75%       0.895994
max       0.943856
Name: JUN - pred, dtype: float64

count    11.000000
mean      0.468175
std       0.204785
min       0.216255
25%       0.284291
50%       0.494281
75%       0.630633
max       0.768691
Name: APOC1 - blood, dtype: float64

count    14.000000
mean      0.502017
std       0.178442
min       0.292091
25%       0.352921
50%       0.515792
75%       0.644921
max       0.798116
Name: APOC1 - pred, dtype: float64

# ZDHHC12 - CCL18

In [26]:
process_genes("ZDHHC12", "CCL18")

count    8.000000
mean     0.065802
std      0.018617
min      0.046841
25%      0.051773
50%      0.062190
75%      0.074195
max      0.099615
Name: ZDHHC12 - blood, dtype: float64

count    9.000000
mean     0.116013
std      0.094682
min      0.029034
25%      0.043078
50%      0.090894
75%      0.166666
max      0.326221
Name: ZDHHC12 - pred, dtype: float64

count    15.000000
mean      0.787977
std       0.032952
min       0.739322
25%       0.763964
50%       0.784812
75%       0.806258
max       0.855855
Name: CCL18 - blood, dtype: float64

count    15.000000
mean      0.699903
std       0.132721
min       0.359589
25%       0.674450
50%       0.727622
75%       0.782909
max       0.901092
Name: CCL18 - pred, dtype: float64

# RASSF2 - CYTIP

In [27]:
process_genes("RASSF2", "CYTIP")

count    16.000000
mean      0.749572
std       0.074465
min       0.626071
25%       0.698348
50%       0.754663
75%       0.798552
max       0.901809
Name: RASSF2 - blood, dtype: float64

count    13.000000
mean      0.750906
std       0.054080
min       0.690023
25%       0.708210
50%       0.742457
75%       0.776490
max       0.882205
Name: RASSF2 - pred, dtype: float64

count    16.000000
mean      0.839056
std       0.074735
min       0.626071
25%       0.812995
50%       0.864356
75%       0.888642
max       0.912679
Name: CYTIP - blood, dtype: float64

count    15.000000
mean      0.843293
std       0.044863
min       0.759377
25%       0.819578
50%       0.843218
75%       0.869837
max       0.914080
Name: CYTIP - pred, dtype: float64

# MYOZ1 - TNNI2

In [28]:
process_genes("MYOZ1", "TNNI2")

count    10.000000
mean      0.171077
std       0.103562
min       0.091422
25%       0.102670
50%       0.113056
75%       0.240818
max       0.367188
Name: MYOZ1 - blood, dtype: float64

count    10.000000
mean      0.108229
std       0.003438
min       0.105677
25%       0.106864
50%       0.106883
75%       0.106883
max       0.115654
Name: MYOZ1 - pred, dtype: float64

count    14.000000
mean      0.219337
std       0.083911
min       0.102468
25%       0.179536
50%       0.221226
75%       0.230911
max       0.436661
Name: TNNI2 - blood, dtype: float64

count    15.000000
mean      0.114366
std       0.003559
min       0.104635
25%       0.114070
50%       0.114303
75%       0.114303
max       0.122063
Name: TNNI2 - pred, dtype: float64

# PYGM - TPM2

In [29]:
process_genes("PYGM", "TPM2")

count    12.000000
mean      0.042046
std       0.032558
min       0.023572
25%       0.023964
50%       0.031966
75%       0.040830
max       0.139322
Name: PYGM - blood, dtype: float64

count    7.000000
mean     0.017173
std      0.009352
min      0.011840
25%      0.012748
50%      0.013268
75%      0.015905
max      0.037800
Name: PYGM - pred, dtype: float64

count    16.000000
mean      0.560692
std       0.154801
min       0.046868
25%       0.537426
50%       0.560244
75%       0.630097
max       0.801900
Name: TPM2 - blood, dtype: float64

count    16.000000
mean      0.277942
std       0.088114
min       0.012290
25%       0.266644
50%       0.275151
75%       0.296155
max       0.466351
Name: TPM2 - pred, dtype: float64